# WEBSCRAPING PROJECT


![WEB IMAGE](web-scraping-with-python.png)

# INTRODUCTION
Webscrapping is the process of extracting data from a website and tranforming this data to a more useful structure that can 
uderstood by a user or an API.
Webscrapping is one of the recquired skills  in the data collection stage and therefore this project is a simple task that i  
have taken to understand webscrapping.
It involves scrapping wikipedia pages and extracting some information which is then stored in a json file.


## IMPORTING PACKAGES
In this stage I import all the necessary packages that i need to work out this project.

In [2]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime
from bs4 import BeautifulSoup as bs


### REGIONS
In this first section, i scrape through the wikipedia website containing regions in uganda and i save the result in a 
json file containing 
- id:
- Region:
- area:
- chief_town:


In [3]:
url1 = 'https://en.wikipedia.org/wiki/Regions_of_Uganda'
url1_text = requests.get('https://en.wikipedia.org/wiki/Regions_of_Uganda').text
soup = bs(url1_text, 'html.parser')
table = soup.find('table', class_ = 'wikitable sortable')
headers = [th.text.strip() for th in table.tr.find_all('th')]

lst_data = []
for index,row in enumerate(table.tbody.find_all('tr')[1:5]):
    data = [td.text.strip() for td in row.find_all('td')]
    lst_data.append(data)
    
dirty_df = pd.DataFrame(data=lst_data, columns=headers)
dirty_df.drop(columns=['Population(Census 1991)','Population(Census 2002)','Population(Census 2014)','Number of Districts'],inplace=True)
Region_df = dirty_df.reset_index().rename(columns = ({'index':'Reg_ID'}))
Region_df.set_index('Reg_ID',inplace = True)
regions = Region_df.to_json(orient = 'records')

with open('regions.json','w+') as f:
    f.write(regions)
    


### DISTRICTS
In this section, i scrape through a wikipedia page containing a number of tables on districts in Uganda and save the data
to a json file containing:
- id
- name
- region_id
- 2014_population
- 2023_population_est
- area

In [18]:
url2 = 'https://en.wikipedia.org/wiki/Districts_of_Uganda'
url2_text = requests.get('https://en.wikipedia.org/wiki/Districts_of_Uganda').text
soup = bs(url2_text,'html.parser')
dis_data = []
for table in soup.find_all('table','wikitable sortable'):
    headerss = [th.text.strip() for th in table.tbody.tr.find_all('th')]
    for index,row in enumerate(table.tbody.find_all('tr')[1:-1]):
        data = [td.text.strip() for td in row.find_all('td')]
        dis_data.append(data)
dirty_df2 = pd.DataFrame(data =dis_data,columns =headerss)
dirty_df2.rename(columns={'Map':'District_ID'},inplace = True)
dirty_df2['District_ID'] = dirty_df2['District_ID'].astype('int')
dirty_df2.sort_values(by='District_ID',ascending = True, inplace = True, ignore_index = True)
dirty_df2
def Region_Identification(district):
    if district <=38 :
        return '3'
    if district == 39 or district <= 75:
        return '2'
    if district == 76 or district <= 101:
        return '0'
    if district > 101:
        return '1'
        
dirty_df2['Reg_ID'] = dirty_df2['District_ID'].apply(Region_Identification)
# dirty_df2.set_index('District_ID',inplace = True)
cols =['District_ID','Reg_ID','Pop (2014)','Pop(2023 est.)','Area (km2)']
dirty = dirty_df2[cols]
dirty
districts = dirty.to_json(orient='records')
with open('districts.json','w+') as f:
    f.write(districts)



### INSTITUTIIONS
In this project, i scrape through wikipedia pages of different institutions and save the following information in 
json file.
- id
- name:
- district_id:
- ownership_type: 

In [26]:
url3 = 'https://unche.or.ug/all-academic-programs/'
url3_text = requests.get('https://unche.or.ug/all-academic-programs/').text
soup = bs(url3_text,'html.parser')
insti_table = soup.find('table', id = 'unche-table')
headersss =[th.text.strip() for th in insti_table.tr.find_all('th')]

institution_data = []
for index,row in enumerate(insti_table.tbody.find_all('tr')):
    dara = [td.text.strip() for td in row.find_all('td')]
    institution_data.append(dara)

dirty_df3 = pd.DataFrame(data = institution_data, columns = headersss)
dirty_df3 = dirty_df3.reset_index().rename(columns = ({'index':'id'}))
def check_ownership(University):
    public = ['Bukalasa Agricultural College','Busitema University','Busitema University','Mbale Campus','Busitema University', 'Nagongera Campus',
'Busitema University Pallisa Campus','Butabika Psychiatric Nursing School','Butabika School of Psychiatric Clinical Officers',
'College of Health Sciences','Makerere University','East African School of Aviation','Fisheries Training Institute','Fortportal College of Health Sciences',
'Gulu College of Health Sciences','Gulu University','Health Tutors College Mulago','Institute of Survey and Land Management'
'Jinja School of Nursing and Midwifery Junior Staff College','Kabale University','Kyambogo University','Law Development Centre Kampala',
'Lira University','Makerere University','Makerere University Business School','Makerere University Business School Arua Campus',
'Makerere University Business School, Jinja Campus','Makerere University Business School, Mbarara','Makerere University College of Agriculture and Environmental Sciences',
'Makerere University College of Business and Management Science','Makerere University College of Computing and Information Science',
'Makerere University College of Engineering, design, Art and Technology','Makerere University College of Humanities and Social Sciences',
'Makerere University College of Natural Sciences','Makerere University College of Veterinary Medicine, Animal Resources and Bio security',
'Makerere University, Jinja Campus','Management Training and Advisory Centre (MTAC)','Masaka School of Comprehensive Nursing',
'Mbale College of Health Sciences','Mbale School of Midwifery and Nursing','Mbarara University of Science and Technology','Medical Laboratory Technician’s School',
'Mountains of the Moon University','Mulago Medical Laboratory School','Mulago School of Dispensing /Pharmacy','Mulago School of Nursing and Midwifery',
'Mulago School of Occupational Therapy','Mulago School of Physiotherapy','Mulago School of Radiography','Muni University','Nakawa Vocational Training Institute',
'National Meteorological Training School','National Teachers College, Kabale','National Teachers College, Kaliro','National Teachers College, Mubende',
'National Teachers College, Muni','National Teachers College, Unyama','Non – Commissioned Officer Academy','Nsamizi Training Institute of Social Development',
'Nyabyeya Forestry College, Masindi','Ophthalmic Clinical Officers Training School','Public Health Nurses’ College','School of Hygiene, Mbale',
'Senior Command and Staff College','Soroti School of Comprehensive Nursing','Soroti University','The Uganda Hotel and Tourism Training Institute',
'Uganda College of Commerce, Aduku','Uganda College of Commerce, Kabale','Uganda College of Commerce, Pakwach','Uganda College of Commerce, Soroti',
'Uganda College of Commerce, Tororo','Uganda Cooperative College Kigumba','Uganda Cooperatives College- Tororo','Uganda Institute of Allied Health and Management Science (UIAHMS) Mulago',
'Uganda Institute of Allied Health and Management Sciences','Uganda Institute of Information and Communications Technology','Uganda Management Institute',
'Uganda Management Institute, Gulu Campus','Uganda Management Institute, Mbale Campus','Uganda Management Institute, Mbarara Campus',
'Uganda Military Academy (UMA)','Uganda Petroleum Institute Kigumba','Uganda Technical College, Bushenyi','Uganda Technical College, Elgon',
'Uganda Technical College, Kichwamba','Uganda Technical College, Kyema,','Uganda Technical College, Lira','Uganda Wildlife Research and Training Institute'
]
    if University in public:
        return 'PUBLIC'
    else:
        return 'PRIVATE'
dirty_df3['Ownership_type'] = dirty_df3['Institution Name'].apply(check_ownership)
dirty_df3.drop(['Program Name','Accredited Date','Expiry Date','Status'],axis =1 ,inplace = True)
institutions = dirty_df3.to_json(orient='records')
with open('institutions.json','w+') as f:
    f.write(institutions)


### PROGRAMS 
In this section a i scrape through the wikipedia page containing different programs and save the following information 
in a json file:
- id:
- name:
- level:
- institution_id:
- accredited_day:
- accredited_month:
- accredited_year:
- expiry_day:
- expiry_month:
- expiry_year:
- status

In [56]:
url4 = 'https://unche.or.ug/all-academic-programs/'
url4_text = requests.get('https://unche.or.ug/all-academic-programs/').text
soup = bs(url4_text,'html.parser')
program_table = soup.find('table', id = 'unche-table')
headings =[th.text.strip() for th in program_table.tr.find_all('th')]

program_data = []
for index,row in enumerate(program_table.tbody.find_all('tr')):
    daraa = [td.text.strip() for td in row.find_all('td')]
    program_data.append(daraa)
    
dirty_df4 = pd.DataFrame(data=program_data , columns = headings)
dirty_df4['Accredited Date'] = pd.to_datetime(dirty_df4['Accredited Date'], infer_datetime_format = True)
dirty_df4['Accredited_Day'] =dirty_df4['Accredited Date'].dt.day
dirty_df4['Accredited_Month'] =dirty_df4['Accredited Date'].dt.month
dirty_df4['Accredited_Year'] =dirty_df4['Accredited Date'].dt.year
dirty_df4['Expiry Date'] = pd.to_datetime(dirty_df4['Expiry Date'], infer_datetime_format = True)
dirty_df4['Expiry_Day'] =dirty_df4['Expiry Date'].dt.day
dirty_df4['Expiry_Month'] =dirty_df4['Expiry Date'].dt.month
dirty_df4['Expiry_Year'] =dirty_df4['Expiry Date'].dt.year

def level (program):
    if program[:3] == 'PhD' or ('Doctor' in program):
        return 'PhD'
    elif program[:4] == 'Diploma' or ('Diploma' in program):
        if 'Postgraduate' in program or ('Higher' in program) or ('Advanced' in program):
            return 'Postgraduate'
        return 'Diploma'
    elif  'Bachelor' in program:
        return 'Bachelor'
    elif  'Master' in program:
        return 'Masters'
    elif 'Certificate' in program:
        return 'Certificate'
    else:
        return 'Unknown'
    
dirty_df4 = dirty_df4.reset_index().rename(columns = ({'index': 'Id'}))
dirty_df4['level'] = dirty_df4['Program Name'].apply(level)
dirty_df4.rename(columns = ({'Program Name':'Name','Institution Name':'Institution'}),inplace = True)
dirty_df4.drop(columns =['Accredited Date','Expiry Date'],inplace =True)
colss =['Id', 'Name', 'level','Institution','Accredited_Day','Accredited_Month','Accredited_Year','Expiry_Day','Expiry_Month','Expiry_Year','Status']
dirty3 = dirty_df4[colss]
programs = dirty3.to_json(orient = 'records')
with open('programs.json','w+') as f:
    f.write(programs)

